In [1]:
import os

In [2]:
%pwd

'/Users/tejasredkar/Developer/Chest-Cancer-Prediction-using-ML-Flow-DVC/research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'/Users/tejasredkar/Developer/Chest-Cancer-Prediction-using-ML-Flow-DVC'

In [34]:
from dataclasses import dataclass
from pathlib import Path
import os
import tensorflow as tf
from cnnClassifier.utils.common import read_yaml, create_directories

@dataclass(frozen=True)
class TrainingConfig:
    root_dir: Path
    trained_model_path: Path
    updated_base_model_path: Path
    training_data: Path
    params_epochs: int
    params_batch_size: int
    params_is_augmentation: bool
    params_image_size: list


class ConfigurationManager:
    def __init__(
        self,
        config_filepath="/Users/tejasredkar/Developer/Chest-Cancer-Prediction-using-ML-Flow-DVC/config/config.yaml",
        params_filepath="/Users/tejasredkar/Developer/Chest-Cancer-Prediction-using-ML-Flow-DVC/params.yaml",
    ):
        # Convert string paths to Path objects
        config_filepath = Path(config_filepath)
        params_filepath = Path(params_filepath)

        # Read configuration and parameters
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        # Create necessary directories
        create_directories([self.config.artifacts_root])

    def get_training_config(self) -> TrainingConfig:
        training = self.config.training
        prepare_base_model = self.config.prepare_base_model
        params = self.params
        training_data = os.path.join(self.config.data_ingestion.unzip_dir, "Chest-CT-Scan-data")
        create_directories([Path(training.root_dir)])

        training_config = TrainingConfig(
            root_dir=Path(training.root_dir),
            trained_model_path=Path(training.trained_model_path),
            updated_base_model_path=Path(prepare_base_model.updated_base_model_path),
            training_data=Path(training_data),
            params_epochs=params.EPOCHS,
            params_batch_size=params.BATCH_SIZE,
            params_is_augmentation=params.AUGMENTATION,
            params_image_size=params.IMAGE_SIZE,
        )

        return training_config


class Training:
    def __init__(self, config: TrainingConfig):
        """
        Initializes the Training class with configuration settings.
        """
        self.config = config
        self.model = None
        self.train_generator = None
        self.valid_generator = None

    def load_base_model(self):
        """
        Load the base model from the specified path in the configuration.
        """
        try:
            self.model = tf.keras.models.load_model(self.config.updated_base_model_path)
            print(f"Base model loaded from {self.config.updated_base_model_path}")
        except Exception as e:
            raise ValueError(f"Error loading model from {self.config.updated_base_model_path}: {e}")

    def train_valid_generator(self):
        """
        Set up the training and validation data generators.
        """
        if not self.config.training_data.exists():
            raise FileNotFoundError(f"Training data directory does not exist: {self.config.training_data}")

        datagenerator_kwargs = dict(
            rescale=1.0 / 255,
            validation_split=0.20
        )

        dataflow_kwargs = dict(
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            interpolation="bilinear"
        )

        # Validation generator
        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(**datagenerator_kwargs)
        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory=str(self.config.training_data),
            subset="validation",
            shuffle=False,
            **dataflow_kwargs
        )

        # Training generator with augmentation if enabled
        if self.config.params_is_augmentation:
            train_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
                rotation_range=40,
                horizontal_flip=True,
                width_shift_range=0.2,
                height_shift_range=0.2,
                shear_range=0.2,
                zoom_range=0.2,
                **datagenerator_kwargs
            )
        else:
            train_datagenerator = valid_datagenerator

        self.train_generator = train_datagenerator.flow_from_directory(
            directory=str(self.config.training_data),
            subset="training",
            shuffle=True,
            **dataflow_kwargs
        )

    @staticmethod
    def save_model(path: Path, model: tf.keras.Model):
        """
        Save the trained model to the specified path.
        """
        try:
            model.save(path)
            print(f"Model saved at {path}")
        except Exception as e:
            raise ValueError(f"Error saving model at {path}: {e}")

    def train(self):
        """
        Train the model using the generators and save the trained model.
        """
        if not self.model:
            raise ValueError("Base model is not loaded. Call `load_base_model` first.")

        if not self.train_generator or not self.valid_generator:
            raise ValueError("Data generators are not initialized. Call `train_valid_generator` first.")

        self.steps_per_epoch = self.train_generator.samples // self.train_generator.batch_size
        self.validation_steps = self.valid_generator.samples // self.valid_generator.batch_size

        print(f"Starting training for {self.config.params_epochs} epochs...")
        self.model.compile(
            optimizer=tf.keras.optimizers.Adam(),
            loss="categorical_crossentropy",
            metrics=["accuracy"]
        )

        self.model.fit(
            self.train_generator,
            epochs=self.config.params_epochs,
            steps_per_epoch=self.steps_per_epoch,
            validation_steps=self.validation_steps,
            validation_data=self.valid_generator
        )

        self.save_model(path=self.config.trained_model_path, model=self.model)


# Enable eager execution
tf.config.run_functions_eagerly(True)

# Main execution
try:
    config_manager = ConfigurationManager()
    training_config = config_manager.get_training_config()
    training_instance = Training(config=training_config)

    training_instance.load_base_model()
    training_instance.train_valid_generator()
    training_instance.train()

except Exception as e:
    print(f"Error: {e}")


[2025-01-15 15:14:07,426: INFO: common : yaml file: /Users/tejasredkar/Developer/Chest-Cancer-Prediction-using-ML-Flow-DVC/config/config.yaml loaded successfully]
[2025-01-15 15:14:07,433: INFO: common : yaml file: /Users/tejasredkar/Developer/Chest-Cancer-Prediction-using-ML-Flow-DVC/params.yaml loaded successfully]
[2025-01-15 15:14:07,434: INFO: common : Directory created: artifacts]
[2025-01-15 15:14:07,435: INFO: common : Directory created: artifacts/training]
[2025-01-15 15:14:07,511: WARNING: saving_utils : Compiled the loaded model, but the compiled metrics have yet to be built. `model.compile_metrics` will be empty until you train or evaluate the model.]
Base model loaded from artifacts/prepare_base_model/base_model_updated.h5
Found 68 images belonging to 2 classes.
Found 275 images belonging to 2 classes.
Starting training for 15 epochs...
Epoch 1/15


/opt/anaconda3/envs/cancer-detection/lib/python3.11/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()
/opt/anaconda3/envs/cancer-detection/lib/python3.11/site-packages/tensorflow/python/data/ops/structured_function.py:258: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.
  warnings.warn(


17/17 ━━━━━━━━━━━━━━━━━━━━ 65s 4s/step - accuracy: 0.5963 - loss: 1.2420 - val_accuracy: 1.0000 - val_loss: 0.0622
Epoch 2/15
 1/17 ━━━━━━━━━━━━━━━━━━━━ 1:00 4s/step - accuracy: 0.8750 - loss: 0.3333

/opt/anaconda3/envs/cancer-detection/lib/python3.11/site-packages/keras/src/trainers/epoch_iterator.py:107: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self._interrupted_warning()


17/17 ━━━━━━━━━━━━━━━━━━━━ 8s 270ms/step - accuracy: 0.8750 - loss: 0.3333 - val_accuracy: 1.0000 - val_loss: 0.0661
Epoch 3/15
17/17 ━━━━━━━━━━━━━━━━━━━━ 66s 4s/step - accuracy: 0.8612 - loss: 0.3752 - val_accuracy: 1.0000 - val_loss: 0.0244
Epoch 4/15
17/17 ━━━━━━━━━━━━━━━━━━━━ 8s 270ms/step - accuracy: 1.0000 - loss: 0.1059 - val_accuracy: 1.0000 - val_loss: 0.0248
Epoch 5/15
17/17 ━━━━━━━━━━━━━━━━━━━━ 67s 4s/step - accuracy: 0.9390 - loss: 0.2047 - val_accuracy: 0.9844 - val_loss: 0.0375
Epoch 6/15
17/17 ━━━━━━━━━━━━━━━━━━━━ 8s 284ms/step - accuracy: 0.8750 - loss: 0.2665 - val_accuracy: 0.9844 - val_loss: 0.0385
Epoch 7/15
17/17 ━━━━━━━━━━━━━━━━━━━━ 65s 4s/step - accuracy: 0.8879 - loss: 0.2999 - val_accuracy: 1.0000 - val_loss: 0.0191
Epoch 8/15
17/17 ━━━━━━━━━━━━━━━━━━━━ 8s 266ms/step - accuracy: 0.8750 - loss: 0.2318 - val_accuracy: 1.0000 - val_loss: 0.0161
Epoch 9/15
17/17 ━━━━━━━━━━━━━━━━━━━━ 68s 4s/step - accuracy: 0.9505 - loss: 0.1354 - val_accuracy: 1.0000 - val_loss: 0.